In [8]:
import cv2
import face_recognition as fr
import numpy as np
from PIL import ImageDraw, Image, ImageFont
import pickle

In [9]:
dataset_file = 'dataset\\staff.dat'             # 請自行修改路徑

with open(dataset_file, 'rb') as f:
    d = pickle.load(f)
    face_encodings = d['encode']
    face_names = d['name']
    
print(face_names)

['成龍', '李連杰', '章子怡', '李御玺', '大魔頭']


In [10]:
font_file = "font\\NotoSansCJKtc-Regular.otf"   # 請自行修改路徑
_font = ImageFont.truetype(font_file, 15) 

In [11]:
com_rate = 0.25

In [7]:
cam = cv2.VideoCapture(1)

while True:
    _, frame = cam.read()
    frame = cv2.flip(frame, 1)
    img_arr = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    img_PIL = Image.fromarray(img_arr)
    small_frame = cv2.resize(frame, (0, 0), fx=com_rate, fy=com_rate)

    draw = ImageDraw.Draw(img_PIL)

    face_loc = fr.face_locations(small_frame)

    for top, right, bottom, left in face_loc:
        top    /= com_rate
        right  /= com_rate
        bottom /= com_rate
        left   /= com_rate
        
        draw.rectangle(((left,top),(right,bottom)), width=6, outline='blue')

    img_CV = cv2.cvtColor(np.asarray(img_PIL), cv2.COLOR_RGB2BGR)
    cv2.imshow('Webcam', img_CV)

    if cv2.waitKey(1) & 0xFF == 27: # 按ESC鍵退出
        break

del draw
cam.release()
cv2.destroyAllWindows()

In [14]:
#cam = cv2.VideoCapture(1)
cam = cv2.VideoCapture("test2.mp4")

while True:
    _, frame = cam.read()
    
    #frame = cv2.flip(frame, 1)
    
    small_frame = cv2.resize(frame, (0, 0), fx=com_rate, fy=com_rate)
    
    face_loc = fr.face_locations(small_frame)
    face_enc = fr.face_encodings(small_frame, face_loc)
    
    img_arr = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_PIL = Image.fromarray(img_arr)
    
    draw = ImageDraw.Draw(img_PIL)

    for (top, right, bottom, left), encoding in zip(face_loc, face_enc):
        top    /= com_rate
        right  /= com_rate
        bottom /= com_rate
        left   /= com_rate
        
        draw.rectangle(((left,top),(right,bottom)), outline='blue')

        matches = fr.compare_faces(face_encodings, encoding, tolerance=0.65)
        
        name = "路人甲"

        if True in matches:
            first_match_index = matches.index(True)
            name = face_names[first_match_index]
            
        txt_w, txt_h = draw.textsize(name, font=_font)
        draw.rectangle(((left, bottom + txt_h + 10), (right, bottom)), fill='blue', outline='blue')
        draw.text((left + 8, bottom + 5), name, fill='white', font=_font) # (255, 255, 255, 255)

    img_CV = cv2.cvtColor(np.asarray(img_PIL), cv2.COLOR_RGB2BGR)
    cv2.imshow('Webcam', img_CV)

    if cv2.waitKey(1) & 0xFF == 27: # 按ESC鍵退出
        break
        
del draw
cam.release()
cv2.destroyAllWindows()